Notebook for checking individual layers and the complete model to compare inputs and outputs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

from normalizing_flows.src.realnvp.dataset import CelebADataset
from normalizing_flows.src.glow.model import layers, blocks, glow_flow


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Test individual layers and blocks

In [ ]:
zero_conv2d = layers.Conv2dZeroInit(in_channels=6, out_channels=128)

# check if all zero outputs
out = zero_conv2d(torch.rand((8, 6, 32, 32)))
print(out.shape)
(out == 0).all()

In [ ]:
inv_conv2d = layers.InvertibleConv2d(in_channels=6)

# check if all zero outputs
inp = torch.rand((8, 6, 32, 32))
out, ldj = inv_conv2d(inp)
out = inv_conv2d.inverse(out)
print(out.shape, ldj.shape)
torch.allclose(inp, out, atol=1e-6)

In [ ]:
affine_coupling = layers.AffineCoupling(in_channels=12, hidden_channels=128)

inp = torch.rand((8, 12, 32, 32))
out, ldj = affine_coupling(inp)
out = affine_coupling.inverse(out)
print(out.shape, ldj.shape)
torch.allclose(inp, out, atol=1e-6)

In [ ]:
flow_step = blocks.FlowStep(in_channels=12)

inp = torch.rand((8, 12, 32, 32))
out, ldj = flow_step(inp)
out = flow_step.inverse(out)
print(out.shape, ldj.shape)
torch.allclose(inp, out, atol=1e-6)

In [ ]:
sf_step = blocks.SqueezeFlowStep(in_channels=12, n_flow_steps=2)

inp = torch.rand((8, 12, 32, 32))
out, ldj = sf_step(inp)
out = sf_step.inverse(out)
print(out.shape, ldj.shape)
torch.allclose(inp, out, atol=1e-6)

# Test realnvp model with input images

In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.CenterCrop(size=128),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Load CelebA dataset
ds_train = CelebADataset(
    root='../data',
    split='train',
    download=True,  # if you have trouble downloading the images, download them manually and move the zip file to ../data/celeba/
    transform=transform
)

# Create dataloader
dl_train = DataLoader(
    ds_train,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    collate_fn=ds_train.collate_fn_skip_errors
)

# Print dataset information
print(f"Train dataset size: {len(ds_train)}")
print(f"Number of batches: {len(dl_train)}")
sample_batch = next(iter(dl_train))[0][:32]
print(f"Batch shape: {sample_batch.shape}")


def show_images(images, nrow=8):
    """Display a grid of images."""
    images = images.cpu()
    # images = images * 0.5 + 0.5  # Denormalize
    grid = torchvision.utils.make_grid(images, nrow=nrow)
    plt.figure(figsize=(15, 15))
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis('off')
    plt.show()


show_images(sample_batch)

In [ ]:
model = glow_flow.Glow(in_channels=3, size=32, final_size=4, n_flow_steps=1)

In [ ]:
out, ldj = model(sample_batch)
out = model.inverse(out)
print(out.shape, ldj.shape)
torch.allclose(sample_batch, out, atol=1e-3)

In [ ]:
show_images(out)